## Homework
## PFound
Исходные данные - Yandex Cup 2022 Analytics
- Ссылка - https://yandex.ru/cup/analytics/analysis/ , пример A. Рассчитать pFound
- Данные - https://yadi.sk/d/guqki4UI4hFlXQ
- Формула
$$pFound@K = \sum_{i=1}^{k} pLook[i]\ pRel[i]$$

$$pLook[1] = 1$$

$$pLook[i] = pLook[i-1]\ (1 - pRel[i-1])\ (1 - pBreak)$$

$$pBreak = 0.15$$

**Задача** - написать функцию, которая принимает на вход dataframe (после join), а на выходе дает средний pFound по всем query.
- Запрещается использовать циклы for для расчет метрики (как полностью, так и ее частей).

In [1]:
!wget -O yandex.zip 'https://getfile.dokpub.com/yandex/get/https://yadi.sk/d/guqki4UI4hFlXQ'

--2022-11-20 21:11:28--  https://getfile.dokpub.com/yandex/get/https://yadi.sk/d/guqki4UI4hFlXQ
Resolving getfile.dokpub.com (getfile.dokpub.com)... 142.132.255.217
Connecting to getfile.dokpub.com (getfile.dokpub.com)|142.132.255.217|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://downloader.disk.yandex.ru/disk/5d0f378f64eab627880f57012323b60b15c8ce3abf28a3ca91e675f9c5ce53ef/637ad041/BaOcgBJYwIzg6Pz-7e1ofdyK_uYONPWbQT9-z6oLtSwltEvXgy43DlfMU7yEVKnvtaL8KRTXRD86Bmt6HmA89w%3D%3D?uid=0&filename=yandex_cup_analytics_A.zip&disposition=attachment&hash=r1pg8imUhL3NYuWf3rq5KaD0gfvSHsK5As/ezZIlBlkNpNLJ8xzFforjLrE57Sprq/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&content_type=application%2Fzip&owner_uid=1168973541&fsize=75391&hid=b7d5cb22d0694a3ed087f532f9612929&media_type=compressed&tknv=v2 [following]
--2022-11-20 21:11:29--  https://downloader.disk.yandex.ru/disk/5d0f378f64eab627880f57012323b60b15c8ce3abf28a3ca91e675f9c5ce53ef/637ad041/BaOcgBJYwIzg6Pz-7e1ofdyK_

In [2]:
!unzip yandex.zip

Archive:  yandex.zip
  inflating: hidden_task.zip         
  inflating: open_task.zip           


In [3]:
!unzip open_task.zip

Archive:  open_task.zip
   creating: open_task/
  inflating: open_task/qid_query.tsv  
  inflating: open_task/hostid_url.tsv  
  inflating: open_task/qid_url_rating.tsv  


In [5]:
import pandas as pd

#YANDEX solution

In [6]:
# считываем данные
qid_query = pd.read_csv("open_task/qid_query.tsv", sep="\t", names=["qid", "query"])
qid_url_rating = pd.read_csv("open_task/qid_url_rating.tsv", sep="\t", names=["qid", "url", "rating"])
hostid_url = pd.read_csv("open_task/hostid_url.tsv", sep="\t", names=["hostid", "url"])

In [7]:
# делаем join двух таблиц, чтобы было просто брать url с максимальным рейтингом
qid_url_rating_hostid = pd.merge(qid_url_rating, hostid_url, on="url")


def plook(ind, rels):
    if ind == 0:
        return 1
    return plook(ind-1, rels)*(1-rels[ind-1])*(1-0.15)


def pfound(group):
    max_by_host = group.groupby("hostid")["rating"].max() # максимальный рейтинг хоста
    top10 = max_by_host.sort_values(ascending=False)[:10] # берем топ10 урлов с наивысшим рейтингом
    pfound = 0
    for ind, val in enumerate(top10):
        pfound += val*plook(ind, top10.values)
    return pfound


qid_pfound = qid_url_rating_hostid.groupby('qid').apply(pfound) # группируем по qid и вычисляем pfound
qid_max = qid_pfound.idxmax() # берем qid с максимальным pfound

qid_query[qid_query["qid"] == qid_max]

,qid,query
12,295761,гугл переводчик


In [9]:
qid_url_rating_hostid.head()

,qid,url,rating,hostid
0,10387,http://batman-arkhamcity.ru/,0.00,64
1,10387,http://bigtorrents.org/publ/batman_arkham_city...,0.14,71
2,10387,http://consolelife.ru/xbox-360/6577-o-rossiysk...,0.14,101
3,10387,http://dic.academic.ru/book.nsf/3662736/Batman...,0.00,115
4,10387,http://forum.csmania.ru/viewtopic.php?t=25986,0.14,155


In [11]:
qid_url_rating_hostid.nunique()

qid        20
url       854
rating      5
hostid    643
dtype: int64

In [13]:
qid_url_rating_hostid[qid_url_rating_hostid.qid == 10387]

,qid,url,rating,hostid
0,10387,http://batman-arkhamcity.ru/,0.00,64
1,10387,http://bigtorrents.org/publ/batman_arkham_city...,0.14,71
2,10387,http://consolelife.ru/xbox-360/6577-o-rossiysk...,0.14,101
3,10387,http://dic.academic.ru/book.nsf/3662736/Batman...,0.00,115
4,10387,http://forum.csmania.ru/viewtopic.php?t=25986,0.14,155
5,10387,http://forum.igromania.ru/printthread.php?s=f3...,0.07,156
6,10387,http://forum.sharereactor.ru/showthread.php?t=...,0.00,161
7,10387,http://forums.4gamers.ru/threads/batman-arkham...,0.07,166
8,10387,http://forums.playground.ru/batman_arkham_city...,0.00,169
9,10387,http://forums.playground.ru/batman_arkham_city...,0.07,169


Нужно вывести текст запроса с максимальным значением метрики , посчитанной по топ-10 документов.
Если для запроса есть несколько документов с одним и тем же id хоста — оставить только максимально релевантный документ (а если несколько документов максимально релевантны, выбрать любой).
Документы по запросу сортируются по убыванию релевантности после выбора одного документа для хоста. Если у нескольких документов с разных хостов релевантность одинакова, их порядок может быть произвольным.

In [44]:
# Если для запроса есть несколько документов с одним и тем же id хоста — оставить только максимально релевантный документ 
# (а если несколько документов максимально релевантны, выбрать любой)
max_rel_rating = qid_url_rating_hostid.groupby(['qid', 'hostid'])['rating'].max()
max_rel_rating = max_rel_rating.reset_index()
max_rel_rating

,qid,hostid,rating
0,10387,64,0.00
1,10387,71,0.14
2,10387,101,0.14
3,10387,115,0.00
4,10387,155,0.14
...,...,...,...
674,380923,1063,0.00
675,380923,1065,0.00
676,380923,1105,0.00
677,380923,1168,0.00


In [101]:
# подсчет топ K для каждого запроса по убыванию релевантности после выбора одного документа для хоста
K = 10
top_K = max_rel_rating.sort_values(['qid', 'rating'], ascending=False).groupby(['qid']).head(K)
top_K

,qid,hostid,rating
647,380923,6,0.14
654,380923,179,0.14
662,380923,386,0.14
666,380923,484,0.14
665,380923,462,0.07
...,...,...,...
12,10387,335,0.14
20,10387,696,0.14
21,10387,740,0.14
24,10387,846,0.14


In [102]:
# дальше получим колонки для формулы plook
# добавим ранк 
top_K['rank'] = top_K.groupby('qid').cumcount()
top_K

,qid,hostid,rating,rank
647,380923,6,0.14,0
654,380923,179,0.14,1
662,380923,386,0.14,2
666,380923,484,0.14,3
665,380923,462,0.07,4
...,...,...,...,...
12,10387,335,0.14,5
20,10387,696,0.14,6
21,10387,740,0.14,7
24,10387,846,0.14,8


In [103]:
# (1 - pBreak) -> (1-0.15)
top_K['1 - p_break'] = 1 - 0.15

In [113]:
# (1 - Rel) -> (1-rels[ind-1])
top_K['1 - rating'] = (1 - top_K['rating']).shift(periods=1, fill_value=0)

In [106]:
# но для нулевого rank нужно добавить 1
top_K.loc[top_K['rank'] == 0, ['1 - p_break', '1 - rating']] = 1

In [107]:
top_K.head()

,qid,hostid,rating,rank,1 - p_break,1 - rating
647,380923,6,0.14,0,1.00,1.00
654,380923,179,0.14,1,0.85,0.86
662,380923,386,0.14,2,0.85,0.86
666,380923,484,0.14,3,0.85,0.86
665,380923,462,0.07,4,0.85,0.86


In [108]:
top_K['prod'] = top_K['1 - p_break'] * top_K['1 - rating']

In [109]:
top_K['plook'] = top_K.groupby('qid')['prod'].cumprod()
top_K['pfound'] = top_K['plook'] * top_K['rating']

top_K.head()

,qid,hostid,rating,rank,1 - p_break,1 - rating,prod,plook,pfound
647,380923,6,0.14,0,1.00,1.00,1.000,1.000000,0.140000
654,380923,179,0.14,1,0.85,0.86,0.731,0.731000,0.102340
662,380923,386,0.14,2,0.85,0.86,0.731,0.534361,0.074811
666,380923,484,0.14,3,0.85,0.86,0.731,0.390618,0.054687
665,380923,462,0.07,4,0.85,0.86,0.731,0.285542,0.019988


In [110]:
pfound_q = top_K.groupby('qid')['pfound'].sum()
pfound_q.mean()

0.4603173929969002

In [111]:
def pfound_q_mean(df, K=10, pBreak=0.15):
    max_rel_rating = df.groupby(['qid', 'hostid'])['rating'].max().reset_index()
    top_K = max_rel_rating.sort_values(['qid', 'rating'], ascending=False).groupby(['qid']).head(K)
    top_K['rank'] = top_K.groupby('qid').cumcount()
    top_K['1 - p_break'] = 1 - pBreak
    top_K['1 - rating'] = (1 - top_K['rating']).shift(periods=1, fill_value=0)
    top_K.loc[top_K['rank'] == 0, ['1 - p_break', '1 - rating']] = 1
    top_K['prod'] = top_K['1 - p_break'] * top_K['1 - rating']
    top_K['plook'] = top_K.groupby('qid')['prod'].cumprod()
    top_K['pfound'] = top_K['plook'] * top_K['rating']
    return top_K.groupby('qid')['pfound'].sum().mean()

In [112]:
pfound_q_mean(qid_url_rating_hostid)

0.4603173929969002